In [1]:
!pip install azure-cognitiveservices-vision-computervision
!pip install flask
!pip install Pillow


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.9/173.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from flask import Flask, request, render_template
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials
import requests
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle


In [ ]:
subscription_key = "412d3fd1a3904a0d9b12e90815d18530"
endpoint = "https://cogservices0.cognitiveservices.azure.com/"
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))


In [ ]:
app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def upload_predict():
    if request.method == 'POST':
        file = request.files.get('image')
        if file:
            image_stream = file.read()
            detected_brands = detect_brands(image_stream)
            return render_template('result.html', brands=detected_brands)
    return render_template('index.html')


def detect_brands(image_stream):
    remote_image_features = ["brands"]
    detect_brands_results_remote = computervision_client.analyze_image_in_stream(image_stream, remote_image_features)

    tmp = []
    print("Detecting brands in remote image: ")
    if len(detect_brands_results_remote.brands) == 0:
        print("No brands detected.")
    else:
        for brand in detect_brands_results_remote.brands:
            print("'{}' brand detected with confidence {:.1f}% at location {}, {}, {}, {}".format( \
            brand.name, brand.confidence * 100, brand.rectangle.x, brand.rectangle.x + brand.rectangle.w, \
            brand.rectangle.y, brand.rectangle.y + brand.rectangle.h))
            tmp.append(brand.name)
            plt.gca().add_patch(Rectangle((brand.rectangle.x,brand.rectangle.y),brand.rectangle.w, brand.rectangle.h,linewidth=1,edgecolor='r',facecolor='none'))

    # Instead of printing the detected brands, return them as a list of dictionaries
    detected_brands = []
    if len(detect_brands_results_remote.brands) > 0:
        for brand in detect_brands_results_remote.brands:
            detected_brands.append({
                'name': brand.name,
                'confidence': brand.confidence * 100,
                'rectangle': {
                    'x': brand.rectangle.x,
                    'y': brand.rectangle.y,
                    'w': brand.rectangle.w,
                    'h': brand.rectangle.h
                }
            })

    return detected_brands

if __name__ == '__main__':
    app.run(debug=True)
